In [13]:
from swf import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit,lsq_linear,brute

In [14]:
octvertices = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]])
octfaces = np.array([[1,2,4],[1,3,4],[3,0,4],[0,2,4],[1,3,5],[3,0,5],[0,2,5],[2,1,5]])

layout704=np.array([[1,0,np.pi/2],[1,np.pi/6,np.pi/2],[1,-np.pi/6,np.pi/2],[1,np.pi/2,np.pi/2],[1,-np.pi/2,np.pi/2],[1,3*np.pi/4,np.pi/2],[1,-3*np.pi/4,np.pi/2],[1,np.pi/4,np.pi/4],[1,-np.pi/4,np.pi/4],[1,3*np.pi/4,np.pi/4],[1,-3*np.pi/4,np.pi/4]])
vertices704 = np.apply_along_axis(lambda x: toCartesian(x),1,layout704)
faces704 = np.array([[6,4,10],[10,4,8],[8,4,2],[8,2,0],[8,7,0],[7,0,1],[7,1,3],[7,3,9],[9,3,5],[10,7,9],[10,8,7],[10,6,5],[10,9,5]])

In [15]:
def optimize(coeffs):
    print('.',end='')
    ALPHA, BETA = coeffs
    GAMMA = (1-2*(ALPHA+BETA))/4
    if checkCoeffRelations(ALPHA,BETA,GAMMA):
        mesh = Trimesh(vertices704,faces704,ALPHA=ALPHA,BETA=BETA,GAMMA=GAMMA)
        model = SWF(mesh, n=3)
        return cost(model,1,1)
    else:
        return 100

In [16]:
res = brute(optimize,((0,1),(0,1)),Ns=64)
res

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

array([0.54621833, 0.03684416])

In [5]:
optimize(res)

.

0.0012977979042934223

In [6]:
optimize((1/2,1/8))

.

0.015260126026117612

In [7]:
optimize((1/2,0))

.

0.09529697163859034

In [9]:
optimize((0.61,-0.01515))

.

0.0013032508167596245

In [27]:
(1-2*(0.54621833+0.03684416))/4

-0.041531244999999994

In [21]:
9/200

0.045